In [4]:
import sys
sys.path.append('..')
from tool import *

In [ ]:
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    for year in [2001,2020]:
        ll=[]
        yy=[]
        mm=[]
        v_rb=[]
        v_s=[]
        coords_y=[]
        coords_x=[]
        saz_Ls=[]
        for m in range(1,13):
            v_rb_m=[]
            v_s_m=[]
            rebuilding=rasterio.open(f"/data2/hzy/ssd_hzy/albedo_rebuilding/rebuilding2_albedo_{year}_{m}_{landcover_type}.tif").read(1)
            source=rasterio.open(root_path+f'albedo_rebuilding/source2_{year}_{m}_sw_albedo2.tif').read(1)
            n=0
            print(year,m)
            coord_g=[]
            for x_g in range(int(80151/40)):
                for y_g in range(int(40075/40)):
                    coord_g.append([x_g,y_g])
            random.shuffle(coord_g)
            for x_g,y_g in coord_g:
                y_c,x_c=random.randint(0,20),random.randint(0,20)
                x=x_g*40+x_c
                y=y_g*40+y_c
                v_rebuilding,v_source=rebuilding[y,x],source[y,x]
                if v_rebuilding!=0 and v_source!=0:
                    if np.isnan(v_rebuilding) or np.isnan(v_source) or v_rebuilding==1:
                        continue
                    else:
                        if v_rebuilding>1:
                            saz_Ls.append(1)
                            v_rebuilding-=1
                        else:
                            saz_Ls.append(0)
                        yy.append(year)
                        mm.append(m)
                        ll.append(landcover_type)
                        v_rb_m.append(v_rebuilding)
                        v_s_m.append(v_source)
                        coords_y.append(y)
                        coords_x.append(x)
                        n+=1
                        if n==50000:
                            break
            v_s+=v_s_m
            v_rb+=v_rb_m     
            print(r2_score(v_s_m,v_rb_m))
            print(np.power(mean_squared_error(v_s_m,v_rb_m), 0.5))
        print('------------------------------all------------------------------')
        print(landcover_type)
        print(r2_score(v_s,v_rb))
        print(np.power(mean_squared_error(v_s,v_rb), 0.5))
        pd.DataFrame.from_dict({'year':yy,'month':mm,'y':coords_y,'x':coords_x,'truth':v_s,'rebuilding':v_rb,'sza_L':saz_Ls,'landcover_scheme':ll}).to_csv(f'../data/rebuilding_usemean_val_{year}_{landcover_type}_buffer10km.csv')




In [6]:
fig = plt.figure(figsize=(15, 25), dpi=600)
for landcover_type_i in range(4):
    landcover_type=['LCCS1','LCCS2','LCCS3','IGBP'][landcover_type_i] 
    print(landcover_type)
    kk=[]
    for y in [2001,2020]:
        kk.append(pd.read_csv(f'../data/rebuilding_usemean_val_{y}_{landcover_type}_buffer10km.csv'))
    kkk=pd.concat(kk)
    kkk.to_csv(f'../data/rebuilding_usemean_val_{landcover_type}.csv')
    kkk_70=kkk[kkk.sza_L<1]
    nn=0
    for sza_L in [70,85]:
        if sza_L==70: 
            x,y=kkk_70['rebuilding'],kkk_70['truth']
        else:
            x,y=kkk['rebuilding'],kkk['truth']
        R2 = r2_score(x,y)
        xy = np.vstack([x, y])
        def slope(xs, ys):
            m = (((mean(xs) * mean(ys)) - mean(xs * ys)) / ((mean(xs) * mean(xs)) - mean(xs * xs)))
            b = mean(ys) - m * mean(xs)
            return m, b
        k, b = slope(x, y)
        regression_line = []
        for a in range(100000):
            regression_line.append((k * a/100000) + b)
        MSE = mean_squared_error(y,x)  
        RMSE = np.power(MSE, 0.5)
        R2 = r2_score(y,x)
        norm = ImageNormalize(vmin=0., vmax=30000, stretch=LogStretch())
        ax = fig.add_subplot(4, 2, landcover_type_i*2+nn+1, projection='scatter_density')
        density = ax.scatter_density(x, y, cmap='afmhot_r',norm=norm)
        cbar=fig.colorbar(density, shrink=1, orientation='vertical', extend='both', pad=0.01, aspect=30, label='Frequency')
        cbar.ax.yaxis.set_major_locator(MultipleLocator(10000))
        ax.grid(True, linestyle='--', alpha=0.2)

        plt.text(0.05, 0.95, landcover_type, transform=ax.transAxes,
        fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.5))
        if nn==0:
            plt.annotate(f'Obs.={len(kkk_70)}',(0.73,0.15))
        else:
            plt.annotate(f'Obs.={len(kkk)}',(0.73,0.15))
        plt.annotate('$R^2=%.3f$' % R2,(0.73,0.1))
        plt.annotate('$RMSE=%.3f$' % RMSE,(0.73,0.05))
        plt.xlabel(r'Monthly blue-sky albedo recontructed from ALLUM',fontsize=10)
        plt.ylabel(r'Monthly blue-sky albedo synthesized from MCD43',fontsize=10)
        plt.axis([0, 1, 0,1])
        ax.legend(loc='upper left', frameon = False)
        nn+=1
plt.savefig(f'../figure/rebuilding500_val.pdf',dpi=600)
plt.show()

LCCS1


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


LCCS2


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


LCCS3


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


IGBP


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [8]:
fig = plt.figure(figsize=(15, 15), dpi=600)
width = 0.2
for landcover_type_i in range(4):
    landcover_type=['LCCS1','LCCS2','LCCS3','IGBP'][landcover_type_i] 
    print(landcover_type)
    kk=[]
    for y in [2001,2020]:
        kk.append(pd.read_csv(f'../data/rebuilding_usemean_val_{y}_{landcover_type}_buffer10km.csv'))
    kkk=pd.concat(kk)
    kkk.to_csv(f'../data/rebuilding_usemean_val_{landcover_type}.csv')
    kkk_70=kkk[kkk.sza_L<1]
    rr70=[]
    mm70=[]
    for m in range(1,13):
        kkk1=kkk_70[kkk_70['month']==m]
        rr70.append(r2_score(kkk1['truth'],kkk1['rebuilding']))
        mm70.append(np.sqrt(mean_squared_error(kkk1['truth'],kkk1['rebuilding'])))
    rr85=[]
    mm85=[]
    for m in range(1,13):
        kkk1=kkk[kkk['month']==m]
        rr85.append(r2_score(kkk1['truth'],kkk1['rebuilding']))
        mm85.append(np.sqrt(mean_squared_error(kkk1['truth'],kkk1['rebuilding'])))
    ax = fig.add_subplot(4, 2, landcover_type_i*2+1)
    x = np.arange(1,13)-width/2
    x2=x+width
    plt.grid(b=None, which='major',zorder=0 )
    plt.bar(x,rr70,width=width,color='#FEB2B4',edgecolor='black',label='SZA<70', zorder=100)
    plt.bar(x2,rr85,width=width,color='#496C88',edgecolor='black',label='SZA<85', zorder=100)
    plt.ylim(0.7,1.04)
    plt.text(0.05, 0.95, landcover_type, transform=ax.transAxes,
    fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.5))
    plt.xlabel(r'Month',)
    plt.ylabel(r'$R^2$')
    plt.xticks(range(1,13))
    plt.legend(ncol=2)   
    ax = fig.add_subplot(4, 2, landcover_type_i*2+2)
    width = 0.2
    x = np.arange(1,13)-width/2
    x2=x+width
    plt.grid(b=None, which='major',zorder=0 )
    plt.bar(x,mm70,width=width,color='#FEB2B4',edgecolor='black',label='SZA<70', zorder=100,hatch='//')
    plt.bar(x2,mm85,width=width,color='#496C88',edgecolor='black',label='SZA<85', zorder=100,hatch='//')
    plt.text(0.05, 0.95, landcover_type, transform=ax.transAxes,
    fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.5))
    plt.xlabel(r'Month')
    plt.ylabel(r'RMSE')
    plt.ylim(0,0.09)
    plt.xticks(range(1,13))
    plt.legend(ncol=2)
plt.savefig(f'../figure/rebuilding500_val_bars.pdf',dpi=600)
plt.show()

LCCS1
LCCS2
LCCS3
IGBP
